In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [17]:
#importing files for reading data.
import os

#importing libraries for profiling
from ydata_profiling import ProfileReport


#libraries for performing market basket analysis.
import tensorflow as tf

from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score
# from mlxtend.frequent_patterns import eclat
from mlxtend.frequent_patterns import apriori, association_rules
from tensorflow.keras.models import Sequential
from mlxtend.frequent_patterns import fpgrowth
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from mlxtend.frequent_patterns import association_rules
from sklearn.ensemble import GradientBoostingClassifier

In [18]:
# Importing Data :
order_product_df = pd.read_csv("Input/order_products__prior.csv")
orders_df = pd.read_csv("Input/orders.csv")
products_df = pd.read_csv("Input/products.csv")
aisles_df = pd.read_csv("Input/aisles.csv")
departments_df = pd.read_csv("Input/departments.csv")

# Merge the datasets
merged_data = pd.merge(orders_df, order_product_df, on='order_id')
merged_data = pd.merge(merged_data,products_df,on='product_id')

# Convert the data to a one-hot encoded format
merged = pd.merge(order_product_df, orders_df, on='order_id', how='left')
merged = pd.merge(merged, products_df, on='product_id', how='left')

merged = merged[merged['product_id'].map(merged['product_id'].value_counts()) > 100000]

del order_product_df
del aisles_df
del departments_df
del orders_df
del products_df
del merged_data

In [19]:
def hot_encode(x):
    if(x<= 0):
        return 0
    if(x>= 1):
        return 1

# Convert data to basket format

basket = merged.groupby(['order_id', 'product_name'])['order_id'].count().unstack().reset_index().fillna(0).set_index('order_id')
basket = basket.applymap(hot_encode)

In [20]:
# Apply Apriori algorithm on training set to find frequent itemsets
frequent_itemsets= apriori(basket, min_support=0.01, use_colnames=True)

# Generate association rules from frequent itemsets
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# Saving Output frequent itemsets and association rules
output_path = f"Output/Apriori/frequent_itemsets.csv"
frequent_itemsets.to_csv(output_path, index=False)

output_path = f"Output/Apriori/association_rules.csv"
rules.to_csv(output_path, index=False)

In [21]:
# Printing Frequent Items Set
'''print("Frequent Itemsets:")
for index, itemset in frequent_itemsets.iterrows():
    print(itemset['itemsets'], "Support:", round(itemset['support'],3))'''

'print("Frequent Itemsets:")\nfor index, itemset in frequent_itemsets.iterrows():\n    print(itemset[\'itemsets\'], "Support:", round(itemset[\'support\'],3))'

In [22]:
# printing Association Rules
'''for index, rule in rules.iterrows():
    antecedents = ', '.join(rule['antecedents'])
    consequents = ', '.join(rule['consequents'])
    support = rule['support']
    confidence = rule['confidence']
    lift = rule['lift']
    
    print(f"Rule #{index+1}:")
    print(f"Antecedents: {antecedents}")
    print(f"Consequents: {consequents}")
    print(f"Support: {support}")
    print(f"Confidence: {confidence}")
    print(f"Lift: {lift}")
    print("-----")'''

'for index, rule in rules.iterrows():\n    antecedents = \', \'.join(rule[\'antecedents\'])\n    consequents = \', \'.join(rule[\'consequents\'])\n    support = rule[\'support\']\n    confidence = rule[\'confidence\']\n    lift = rule[\'lift\']\n    \n    print(f"Rule #{index+1}:")\n    print(f"Antecedents: {antecedents}")\n    print(f"Consequents: {consequents}")\n    print(f"Support: {support}")\n    print(f"Confidence: {confidence}")\n    print(f"Lift: {lift}")\n    print("-----")'